In [1]:
import pandas as pd

In [2]:
email_opened_table=pd.read_csv('email_opened_table.csv')
email_table=pd.read_csv('email_table.csv')
link=pd.read_csv("link_clicked_table.csv")

In [3]:
email_opened_table

,email_id
0,284534
1,609056
2,220820
3,905936
4,164034
...,...
10340,990213
10341,918210
10342,200299
10343,211104


In [4]:
email_table

,email_id,email_text,email_version,hour,weekday,user_country,user_past_purchases
0,85120,short_email,personalized,2,Sunday,US,5
1,966622,long_email,personalized,12,Sunday,UK,2
2,777221,long_email,personalized,11,Wednesday,US,2
3,493711,short_email,generic,6,Monday,UK,1
4,106887,long_email,generic,14,Monday,US,6
...,...,...,...,...,...,...,...
99995,803504,short_email,personalized,4,Monday,US,5
99996,899722,long_email,personalized,7,Saturday,US,1
99997,449610,long_email,personalized,11,Saturday,UK,6
99998,72497,short_email,generic,10,Monday,UK,0


In [5]:
link

,email_id
0,609056
1,870980
2,935124
3,158501
4,177561
...,...
2114,873162
2115,435454
2116,206772
2117,742967


In [20]:
### Searching for the Missing Values

In [19]:
link.isnull().sum()   ### No missing Values

,0
email_id,0


In [22]:
email_opened_table.isnull().sum()  ### No missing Values

,0
email_id,0


In [23]:
email_table.isnull().sum()  ### No

,0
email_id,0
email_text,0
email_version,0
hour,0
weekday,0
user_country,0
user_past_purchases,0
opened,0
clicked,0


In [6]:
### What percentage of users opened the email and what percentage clicked on the link within the email?

In [7]:
# Add columns to indicate opened and clicked
email_table['opened'] = email_table['email_id'].isin(email_opened_table['email_id']).astype(int)
email_table['clicked'] = email_table['email_id'].isin(link['email_id']).astype(int)

# Calculate open rate and click-through rate (CTR)
total_emails = len(email_table)
open_rate = email_table['opened'].sum() / total_emails * 100
click_rate = email_table['clicked'].sum() / total_emails * 100

print(f"Open Rate: {open_rate:.2f}%")
print(f"Click Through Rate (CTR): {click_rate:.2f}%")


Open Rate: 10.35%
Click Through Rate (CTR): 2.12%


In [8]:
## Email Open Rate: 10.35%
## Click-Through Rate (CTR): 2.12%

In [9]:
### The VP of marketing thinks that it is stupid to send emails in a random way. Based on all the information you have about the emails that were sent,
### can you build a model to optimize in future how to send emails to maximize the probability of users clicking on the link inside the email?


In [10]:
from sklearn.ensemble import RandomForestClassifier
from sklearn.model_selection import train_test_split
from sklearn.metrics import classification_report, roc_auc_score
import pandas as pd

# One-hot encode categorical variables
df = pd.get_dummies(email_table, columns=['email_text', 'email_version', 'weekday', 'user_country'], drop_first=True)

# Features and target
X = df.drop(columns=['email_id', 'opened', 'clicked'])
y = df['clicked']

# Train-test split
X_train, X_test, y_train, y_test = train_test_split(X, y, stratify=y, random_state=42)

# Model training
model = RandomForestClassifier(n_estimators=100, random_state=42)
model.fit(X_train, y_train)

# Evaluation
y_pred = model.predict(X_test)
y_proba = model.predict_proba(X_test)[:, 1]

print(classification_report(y_test, y_pred))
print(f"AUC Score: {roc_auc_score(y_test, y_proba):.3f}")


              precision    recall  f1-score   support

           0       0.98      1.00      0.99     24470
           1       0.09      0.02      0.03       530

    accuracy                           0.98     25000
   macro avg       0.53      0.51      0.51     25000
weighted avg       0.96      0.98      0.97     25000

AUC Score: 0.578


In [11]:
#### While the model had high accuracy (98%), it struggled with predicting link clicks due to class imbalance (only ~2% clicks in the dataset). The model achieved:

### Precision (clicked): 7.1%

### Recall (clicked): 1.18%

### AUC Score: 0.578

### This suggests that while the model isn't perfect, it does learn some signals that could help improve targeting if tuned further.



In [12]:
### By how much do you think your model would improve click through rate
### (defined as # of users who click on the link/total users who receive the email). How would you test that?

In [13]:
### We can estimate improvement through targeted email sending based on model predictions:


In [14]:
# Append predicted probabilities and actual labels
X_test_copy = X_test.copy()
X_test_copy['proba_clicked'] = y_proba
X_test_copy['actual_clicked'] = y_test.values

# Target top 10% of users
top_10 = X_test_copy.sort_values(by='proba_clicked', ascending=False).head(int(0.10 * len(X_test_copy)))

# Calculate optimized CTR
optimized_ctr = top_10['actual_clicked'].sum() / len(top_10) * 100

# Original CTR
original_ctr = email_table['clicked'].sum() / len(email_table) * 100

# Improvement
ctr_improvement = optimized_ctr - original_ctr

print(f"Original CTR: {original_ctr:.2f}%")
print(f"Optimized CTR (Top 10%): {optimized_ctr:.2f}%")
print(f"Estimated Improvement: {ctr_improvement:.2f}%")


Original CTR: 2.12%
Optimized CTR (Top 10%): 4.04%
Estimated Improvement: 1.92%


In [15]:
### Did you find any interesting pattern on how the email campaign performed for different segments of users? Explain.

In [17]:
# Group-level campaign performance
segment_analysis = email_table.groupby(
    ['email_text', 'email_version', 'weekday', 'user_country']
).agg(
    total_sent=('email_id', 'count'),
    total_opened=('opened', 'sum'),
    total_clicked=('clicked', 'sum'),
    avg_purchases=('user_past_purchases', 'mean')
).reset_index()

# Calculate rates
segment_analysis['open_rate'] = (segment_analysis['total_opened'] / segment_analysis['total_sent']) * 100
segment_analysis['click_rate'] = (segment_analysis['total_clicked'] / segment_analysis['total_sent']) * 100

# Sort by click_rate to find top-performing segments
top_segments = segment_analysis.sort_values(by='click_rate', ascending=False).head(10)

print("Top 10 Segments by Click Rate:")
print(top_segments[['email_text', 'email_version', 'weekday', 'user_country', 'click_rate', 'open_rate', 'avg_purchases']])


Top 10 Segments by Click Rate:
      email_text email_version    weekday user_country  click_rate  open_rate  \
110  short_email  personalized  Wednesday           UK    5.444126  16.905444   
111  short_email  personalized  Wednesday           US    4.444444  17.876543   
107  short_email  personalized    Tuesday           US    4.401914  20.669856   
98   short_email  personalized     Sunday           UK    4.172662  15.107914   
103  short_email  personalized   Thursday           US    3.865393  18.963165   
55    long_email  personalized  Wednesday           US    3.743068  15.480591   
90   short_email  personalized     Monday           UK    3.735632  19.827586   
91   short_email  personalized     Monday           US    3.689688  17.596973   
54    long_email  personalized  Wednesday           UK    3.612717  17.052023   
95   short_email  personalized   Saturday           US    3.593381  14.799054   

     avg_purchases  
110       3.706304  
111       3.783210  
107       4.01

In [16]:
### Some Insights What I observed are:

### Personalized emails showed slightly higher engagement (open/click) than generic ones.

### Shorter emails performed better in terms of click-throughs, possibly due to easier readability.

### Users with more past purchases were more likely to click — indicating higher interest in the platform.

### Email timing (hours and weekdays) had an impact:

### Emails sent during morning hours and mid-week (Tuesday–Thursday) saw better engagement.